# Exercise 5 [Direct Transcription Methods]

In [56]:
# a few packages we need to import

%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.animation as animation
import IPython
import itertools

import cvxopt

# Using a quadratic program solver

In this exercise we will learn how to use a QP solver to solve LQ problems as in the model predictive control scheme presented in the class. Here we will use the quadratic program solver from cvxopt (https://cvxopt.org/). The installation instructions for the solver can be found here: https://cvxopt.org/install/index.html

A tutorial on how to use the QP solver can be found here: https://cvxopt.org/examples/tutorial/qp.html

And the function definition to use the QP solver can be found here: https://cvxopt.org/userguide/coneprog.html#quadratic-programming

As a first example, we see how a simple QP can be solved. Consider the following Quadratic Program
$$\min_x \frac{1}{2} x^T \begin{bmatrix} 1 & 0\\ 0 & 1 \end{bmatrix} x + \begin{bmatrix} -1 \\ -1 \end{bmatrix}^T x$$
subject to: $$ x_i > 0 \qquad \forall x_i$$
and
$$ \begin{bmatrix} 1 \\ 1 \end{bmatrix}^T x = 1$$

The first constraint just says that each element of $x$ needs to be positive and the second constraint says that the sum of the elements of $x$ needs to be equal to 1. Below is a way to use cvxopt to solve the problem.

In [106]:
# the qp solve of cvxopt accepts problems of the form 0.5 xT P x + qT x, subject to Gx <= h, Ax=b
# in our case we have
P = np.array([[1.,0],[0,1]]) # be careful to write 1. instead of 1 in Python 2.0
q = np.array([[-1.],[-1.]])
G = np.array([[-1.,0],[0,-1.]])
h = np.array([[0.],[0]])
A = np.array([[1.,1]])
b = np.array([[1.]])

# we need to convert numpy arrays into the matrix structures understood by cvxopt
# you could also construct directly matrix objects without passing through numpy 
# here we demonstrate how to go from numpy to cvxopt.matrix because we have used numpy arrays thus far
P = cvxopt.matrix(P)
q = cvxopt.matrix(q)
G = cvxopt.matrix(G)
h = cvxopt.matrix(h)
A = cvxopt.matrix(A)
b = cvxopt.matrix(b)

# now we can just solve the qp by calling the qp function from cvxopt
sol = cvxopt.solvers.qp(P,q,G,h,A,b)
# we convert the solution as a numpy array back
x = np.array(sol['x'])

print('The solution of the problem is x=')
print(x)

     pcost       dcost       gap    pres   dres
 0: -7.5000e-01 -1.7500e+00  1e+00  0e+00  2e+00
 1: -7.5000e-01 -7.6000e-01  1e-02  2e-16  2e-02
 2: -7.5000e-01 -7.5010e-01  1e-04  3e-16  1e-04
 3: -7.5000e-01 -7.5000e-01  1e-06  0e+00  2e-06
 4: -7.5000e-01 -7.5000e-01  1e-08  0e+00  2e-08
Optimal solution found.
The solution of the problem is x=
[[0.5]
 [0.5]]


If we wanted to solve the same problem without the constraint that the sum of the vector components should be equal to 1, we could do it simply by omiting the A and b inputs

In [107]:
# now we can just solve the qp by calling the qp function from cvxopt
sol = cvxopt.solvers.qp(P,q,G,h)
# we convert the solution as a numpy array back
x = np.array(sol['x'])

print('The solution of the problem is x=')
print(x)

     pcost       dcost       gap    pres   dres
 0: -7.5000e-01 -1.7500e+00  1e+00  0e+00  1e+00
 1: -9.4382e-01 -9.5908e-01  2e-02  2e-16  2e-01
 2: -1.0000e+00 -1.0018e+00  2e-03  0e+00  6e-17
 3: -1.0000e+00 -1.0000e+00  2e-05  0e+00  2e-17
 4: -1.0000e+00 -1.0000e+00  2e-07  3e-16  9e-17
Optimal solution found.
The solution of the problem is x=
[[1.00000009]
 [1.00000009]]


# Direct transcription for the car model

Consider a 2 dimensional car model
$$ x_{n+1} = \begin{bmatrix} 1 & \Delta t & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 1 & \Delta t \\ 0 & 0 & 0 & 1 \end{bmatrix} x_n + \begin{bmatrix} 0 & 0\\ \Delta t & 0 \\ 0 & 0 \\ 0 & \Delta t \end{bmatrix} u_n$$

where the first element of $x_n$ is the car horizontal position, the second element is the horizontal velocity, the third element the vertical position and the last element the vertical velocity. $u_n$ contains the horizontal and vertical acceleration as control inputs.

We would like to get the car from initial position (0,0) to the final goal (5,2) with 0 desired final velocities under the constraints that the maximum acceleration is limited to 5 in each direction. Here we will use $\Delta t = 0.1$ and an horizon length of 100 steps.

## Question 1
Write an optimal control problem (i.e. with a cost and constraints) that could find the desired behavior for the car  (you may encode the desired goal in the cost function).

## Question 2
We would like to solve the problem using the transcription method shown in the class. 
1. Write a function that finds a  solution to the problem (i.e. a control trajectory and the associated car states).
2. Find weights in your cost function such that the behavior resembles a  bang-bang controller (i.e. minimum time). Plot the resulting control input and state trajectory and use the animation function below to make a movie of the behavior
3. Find weights that reduce the bang bang behavior smoother. As before, plot states and controls and show an animation

## Question 3
Now assume that we add an additional constraint saying that at all times, the sum of the horizontal and vertical velocities of the car cannot be greated than 1. 
1. Write the new optimal control problem.
2. Write a novel function that solves this new problem.
3. How do the trajectories change compared to the case without the constraint? Analyze the resulting behavior, could you have found it manually? (as before, plot trajectories and show an animation)


In [103]:
def solve_direct_transcription(A,B,Q,R,x0,horizon_length, u_max, x_des):
    """
    Here we want to find the optimal control path using a direct transcription approach
    
    Inputs: 
    A,B system dynamics
    Q,R: numpy arrays for the quadratic cost
    x_des: the desired goal position
    u_max: the maximum acceleration
    horizon_length: the number of steps
    
    returns: the state and control trajectories
    """    
    
    # the lenght of the state vector for the full trajectory is
    num_states = x0.size*horizon_length
    # the length of the control vector for the full trajectory is
    num_control = u_len*horizon_length
    # the total number of variables is
    num_vars = num_states + num_control


    ######################################################################
    ###  here you need to construct the problem as a quadratic program ###
    ######################################################################
    ## P: num_vars * num_vars will contain the cost quadratic term
    ## q: num_vars * 1 will contain the cost linear term
    ## G: will contain the inequality contraints (max acceleration)
    ## h: will contain the max and min acceleration
    ## A and b: will contain the equality constraints associtated to the system dynamics
    
    # once the problem is constructed we can call the solver
    sol = cvxopt.solvers.qp(P,q,G,h,A,b)
    x = np.array(sol['x'])
    
    ## we assume that the problem was constructed with the states first (x0,x1,...)
    ## and then the control inputs (u0, u1, ...)
    
    # we extract the control trajectory as a 2 * horizon_length array
    u = (x[num_states:].reshape(horizon_length,2)).transpose()
    
    # we extract the state trajectory and add the initial condition
    x = x[0:num_states]
    x = np.vstack((x0, x.reshape([horizon_length, x0.size]))).transpose()

    return x, u
        

In [99]:
def animate_car(x, goal):
    """
    This function makes an animation showing the behavior of the car
    takes as input the result of the solve_direct_transcription function 
    and the desired goal (x,y position)
    """
    
    #subsample
    plotx = x[:,:]
    
    fig = matplotlib.figure.Figure(figsize=[8,8])
    matplotlib.backends.backend_agg.FigureCanvasAgg(fig)
    ax = fig.add_subplot(111, autoscale_on=False, xlim=[-2,5.5], ylim=[-2,5.5])
    ax.grid()
    
    list_of_lines = []
    
    #create the cart pole
    line, = ax.plot([], [], 'k', lw=2)
    list_of_lines.append(line)
    line, = ax.plot([], [], 'k', lw=2)
    list_of_lines.append(line)
    line, = ax.plot([], [], 'k', lw=2)
    list_of_lines.append(line)
    line, = ax.plot([], [], 'k', lw=2)
    list_of_lines.append(line)
    line, = ax.plot([], [], 'ob', lw=2)
    list_of_lines.append(line)
    
    car_length = 0.05
    car_height = 0.05
    
    def animate(i):
        for l in list_of_lines: #reset all lines
            l.set_data([],[])
        
        x_back = plotx[0,i] - car_length
        x_front = plotx[0,i] + car_length
        y_up = plotx[2,i] + car_height
        y_down = plotx[2,i] - car_height
        
        list_of_lines[0].set_data([x_back, x_front], [y_down, y_down])
        list_of_lines[1].set_data([x_front, x_front], [y_down, y_up])
        list_of_lines[2].set_data([x_back, x_front], [y_up, y_up])
        list_of_lines[3].set_data([x_back, x_back], [y_down, y_up])
        list_of_lines[4].set_data([goal[0],goal[0]],[goal[1],goal[1]])
        
        return list_of_lines
    
    def init():
        return animate(0)


    ani = animation.FuncAnimation(fig, animate, np.arange(0, len(plotx[0,:])),
        interval=100, blit=True, init_func=init)
    plt.close(fig)
    plt.close(ani._fig)
    IPython.display.display_html(IPython.core.display.HTML(ani.to_html5_video()))